<a href="https://colab.research.google.com/github/Phionanamugga/AI_Powered_Disease_Detection/blob/main/heart_disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing datasets
import cv2
import os
import numpy as np
from albumentations import Compose, Resize, Normalize, HorizontalFlip
from albumentations.pytorch import ToTensorV2


In [2]:
transform = Compose([
    Resize(224, 224),
    HorizontalFlip(p=0.5),
    Normalize(),
    ToTensorV2()
])

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    augmented = transform(image=image)
    return augmented['image']

# Usage example
# tensor_image = preprocess_image('data/raw/00000001_000.png')

In [3]:
# src/model/model.py
import torch
import torch.nn as nn
from torchvision import models

class DiseaseDetectionModel(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        self.base_model = models.efficientnet_b0(pretrained=True)
        self.base_model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.base_model.classifier[1].in_features, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.base_model(x)

In [4]:
# src/train/train.py
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_auc_score
import torch.nn as nn
import torch.optim as optim

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = preprocess_image(self.image_paths[idx])
        label = torch.tensor(self.labels[idx]).float()
        return image, label

def train_model(model, dataloader, epochs=10):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    model.train()

    for epoch in range(epochs):
        epoch_loss = 0
        all_labels = []
        all_preds = []

        for images, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            all_labels.extend(labels.numpy())
            all_preds.extend(outputs.squeeze().detach().numpy())

        auc = roc_auc_score(all_labels, all_preds)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(dataloader):.4f}, AUC: {auc:.4f}")

    torch.save(model.state_dict(), 'models/disease_detection.pth')
